In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

df= pd.read_csv('C://Users//User//Desktop//MSc Westminster//Dissertation//DataSets//Heart_Disease_Indicators.csv')

In [2]:
from sklearn.model_selection import train_test_split

target_size = 50000

num_class_0 = int(target_size * df['HeartDiseaseorAttack'].value_counts(normalize=True)[0])
num_class_1 = target_size - num_class_0

df_class_0 = df[df['HeartDiseaseorAttack'] == 0]
df_class_1 = df[df['HeartDiseaseorAttack'] == 1]

df_class_0_sampled = df_class_0.sample(n=num_class_0, random_state=15)
df_class_1_sampled = df_class_1.sample(n=num_class_1, random_state=15)

df_sampled = pd.concat([df_class_0_sampled, df_class_1_sampled])

df_sampled = df_sampled.sample(frac=1, random_state=15).reset_index(drop=True)
df= df_sampled
print(df['HeartDiseaseorAttack'].value_counts(normalize=True))

0    0.90582
1    0.09418
Name: HeartDiseaseorAttack, dtype: float64


In [3]:
df.drop_duplicates(inplace= True)
df.shape

(48050, 22)

In [4]:
df= df[df['Diabetes'] != 1].copy()
df.loc[df['Diabetes'] == 2, 'Diabetes'] = 1
print(df['Diabetes'].value_counts())

0    40175
1     6968
Name: Diabetes, dtype: int64


In [5]:
categorical_columns= ['HighBP', 'HighChol', 'CholCheck', 'Smoker', 'Stroke', 'Diabetes', 'PhysActivity',
                      'Fruits', 'Veggies', 'HvyAlcoholConsump', 'AnyHealthcare', 'NoDocbcCost', 'DiffWalk',
                      'Sex', 'Age', 'Education', 'Income'
                     ]
df[categorical_columns]= df[categorical_columns].astype(str)
df= pd.get_dummies(df, columns= categorical_columns, drop_first= True)

In [6]:
X= df.drop(['HeartDiseaseorAttack'], axis= 1)
y= df['HeartDiseaseorAttack']

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size= 0.2, random_state= 15, stratify= y)

continuous_columns = ['BMI', 'GenHlth', 'MentHlth', 'PhysHlth']

from sklearn.preprocessing import StandardScaler
ss= StandardScaler()
X_train= ss.fit_transform(X_train)
X_test= ss.transform(X_test)

In [8]:
from imblearn.combine import SMOTETomek

smt= SMOTETomek(random_state= 15)
X_sm_tl, y_sm_tl= smt.fit_resample(X_train, y_train)

In [9]:
def backward_elimination(data, target, significance_level= 0.05):
    features= data.columns.tolist()
    while len(features) > 0:
        model= sm.Logit(target, data[features])
        result= model.fit(disp=0)
        p_values= result.pvalues
        max_p_value= p_values.max()
        if max_p_value > significance_level:
            excluded_feature= p_values.idxmax()
            print(f"Excluding {excluded_feature} with p-value {max_p_value}")
            features.remove(excluded_feature)
        else:
            break
    return data[features]

In [10]:
import statsmodels.api as sm

X_sm_tl_df= pd.DataFrame(X_sm_tl, columns= X.columns)
X_test_df= pd.DataFrame(X_test, columns= X.columns)

selected_features= backward_elimination(X_sm_tl_df, y_sm_tl)
X_sm_tl_selected= X_sm_tl_df[selected_features.columns]
X_sm_tl_test_selected= X_test_df[selected_features.columns]

Excluding Age_3 with p-value 0.8916461102393793
Excluding Age_4 with p-value 0.776848514514908
Excluding MentHlth with p-value 0.45285105957966953
Excluding Age_2 with p-value 0.27374642021734785
Excluding Age_5 with p-value 0.12734869261596482
Excluding Income_2 with p-value 0.10389922013374969
Excluding Income_3 with p-value 0.2398832276860493
Excluding Income_4 with p-value 0.06073589824560146
Excluding Fruits_1 with p-value 0.059658884571611774


--- LogisticRegression ---

In [11]:
from sklearn.metrics import confusion_matrix, classification_report, precision_score, roc_auc_score, accuracy_score
from sklearn.linear_model import LogisticRegression
lr= LogisticRegression(random_state= 15)
lr.fit(X_sm_tl_selected, y_sm_tl)
y_pred_lr= lr.predict(X_sm_tl_test_selected)
y_pred_prob_lr= lr.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_lr, labels= lr.classes_)
print(classification_report(y_test, y_pred_lr, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_lr, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_lr))
print('AUC:', roc_auc_score(y_test, y_pred_prob_lr))

              precision    recall  f1-score   support

           0       0.97      0.75      0.85      8517
           1       0.26      0.80      0.39       912

    accuracy                           0.76      9429
   macro avg       0.61      0.78      0.62      9429
weighted avg       0.90      0.76      0.80      9429

Precision: 0.25637443241355223
Accuracy: 0.7553293032134903
AUC: 0.8517667966440698


--- DecisionTreeClassifier ---

In [12]:
from sklearn.tree import DecisionTreeClassifier
dt= DecisionTreeClassifier(random_state=15)
dt.fit(X_sm_tl_selected, y_sm_tl)
y_pred_dt= dt.predict(X_sm_tl_test_selected)
y_pred_prob_dt= dt.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_dt, labels= dt.classes_)
print(classification_report(y_test, y_pred_dt, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_dt, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_dt))
print('AUC:', roc_auc_score(y_test, y_pred_prob_dt))

              precision    recall  f1-score   support

           0       0.92      0.90      0.91      8517
           1       0.24      0.28      0.26       912

    accuracy                           0.84      9429
   macro avg       0.58      0.59      0.58      9429
weighted avg       0.86      0.84      0.85      9429

Precision: 0.23572744014732966
Accuracy: 0.8424011029801676
AUC: 0.5909738829873792


--- Tuned - DecisionTreeClassifier ---

In [13]:
from sklearn.model_selection import GridSearchCV
param_grid= {
             'max_depth': [None, 10, 20, 30, 40, 50],
             'min_samples_split': [2, 10, 20],
             'min_samples_leaf': [1, 5, 10],
             'max_features': [None, 'sqrt', 'log2'],
             'criterion': ['gini', 'entropy']
            }

gs_dt= GridSearchCV(estimator= dt, param_grid= param_grid, cv= 5, scoring= 'precision')
gs_dt.fit(X_sm_tl, y_sm_tl)

print("Best Parameters:", gs_dt.best_params_)
print("Best Precision Score:", gs_dt.best_score_)

Best Parameters: {'criterion': 'entropy', 'max_depth': None, 'max_features': None, 'min_samples_leaf': 10, 'min_samples_split': 2}
Best Precision Score: 0.9408415474451101


In [14]:
tuned_dt= gs_dt.best_estimator_
tuned_dt.fit(X_sm_tl_selected, y_sm_tl)
y_pred_tuned_dt= tuned_dt.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_dt= tuned_dt.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_dt, labels= tuned_dt.classes_)
print(classification_report(y_test, y_pred_tuned_dt, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_dt, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_dt))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_dt))

              precision    recall  f1-score   support

           0       0.92      0.94      0.93      8517
           1       0.29      0.22      0.25       912

    accuracy                           0.87      9429
   macro avg       0.60      0.58      0.59      9429
weighted avg       0.86      0.87      0.86      9429

Precision: 0.2861271676300578
Accuracy: 0.8718846113055467
AUC: 0.7229181343195961


--- KNeighborsClassifier ---

In [15]:
from sklearn.neighbors import KNeighborsClassifier
knn= KNeighborsClassifier()
knn.fit(X_sm_tl_selected, y_sm_tl)
y_pred_knn= knn.predict(X_sm_tl_test_selected)
y_pred_prob_knn= knn.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_knn, labels= knn.classes_)
print(classification_report(y_test, y_pred_knn, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_knn, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_knn))
print('AUC:', roc_auc_score(y_test, y_pred_prob_knn))

              precision    recall  f1-score   support

           0       0.95      0.79      0.86      8517
           1       0.22      0.58      0.32       912

    accuracy                           0.76      9429
   macro avg       0.58      0.68      0.59      9429
weighted avg       0.88      0.76      0.81      9429

Precision: 0.2230246389124894
Accuracy: 0.7649803796797116
AUC: 0.7398906392581195


--- Tuned - KNeighborsClassifier ---

In [16]:
param_grid= {
             'n_neighbors': np.arange(1,40),
             'weights': ['uniform', 'distance'],
             'metric': ['euclidean', 'manhattan', 'minkowski']
            }

gs_knn= GridSearchCV(estimator= knn, param_grid= param_grid, cv=5, scoring= 'precision')
gs_knn.fit(X_sm_tl, y_sm_tl)
print("Best Parameters:", gs_knn.best_params_)
print("Best Precision Score:", gs_knn.best_score_)

Best Parameters: {'metric': 'manhattan', 'n_neighbors': 2, 'weights': 'uniform'}
Best Precision Score: 0.9311578759522989


In [17]:
tuned_knn= gs_knn.best_estimator_
tuned_knn.fit(X_sm_tl_selected, y_sm_tl)

y_pred_tuned_knn= tuned_knn.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_knn= tuned_knn.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_knn, labels= tuned_knn.classes_)
print(classification_report(y_test, y_pred_tuned_knn, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_knn, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_knn))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_knn))

              precision    recall  f1-score   support

           0       0.92      0.94      0.93      8517
           1       0.25      0.19      0.21       912

    accuracy                           0.87      9429
   macro avg       0.58      0.56      0.57      9429
weighted avg       0.85      0.87      0.86      9429

Precision: 0.24635568513119532
Accuracy: 0.8663697104677061
AUC: 0.6211567448178976


--- GaussianNB ---

In [18]:
from sklearn.naive_bayes import GaussianNB
nb= GaussianNB()
nb.fit(X_sm_tl_selected, y_sm_tl)
y_pred_nb= nb.predict(X_sm_tl_test_selected)
y_pred_prob_nb= nb.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_nb, labels= nb.classes_)
print(classification_report(y_test, y_pred_nb, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_nb, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_nb))
print('AUC:', roc_auc_score(y_test, y_pred_prob_nb))

              precision    recall  f1-score   support

           0       0.96      0.71      0.82      8517
           1       0.22      0.76      0.34       912

    accuracy                           0.71      9429
   macro avg       0.59      0.73      0.58      9429
weighted avg       0.89      0.71      0.77      9429

Precision: 0.21665623043206012
Accuracy: 0.7113161522961078
AUC: 0.7862668625597102


--- SVM ---

In [19]:
from sklearn.svm import SVC
svc= SVC(kernel= 'rbf',probability= True, gamma= 1, random_state=15)
svc.fit(X_sm_tl_selected, y_sm_tl)
y_pred_svc= svc.predict(X_sm_tl_test_selected)
y_pred_prob_svc= svc.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_svc, labels= svc.classes_)
print(classification_report(y_test, y_pred_svc, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_svc, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_svc))
print('AUC:', roc_auc_score(y_test, y_pred_prob_svc))

              precision    recall  f1-score   support

           0       0.91      0.96      0.93      8517
           1       0.16      0.06      0.09       912

    accuracy                           0.88      9429
   macro avg       0.53      0.51      0.51      9429
weighted avg       0.83      0.88      0.85      9429

Precision: 0.15598885793871867
Accuracy: 0.8770813447873581
AUC: 0.7147559241681755


--- Random Forest ---

In [20]:
from sklearn.ensemble import RandomForestClassifier
rf= RandomForestClassifier(random_state=15)
rf.fit(X_sm_tl_selected, y_sm_tl)
y_pred_rf= rf.predict(X_sm_tl_test_selected)
y_pred_prob_rf= rf.predict_proba(X_sm_tl_test_selected)[:,1]

In [21]:
cm= confusion_matrix(y_test, y_pred_rf, labels= rf.classes_)
print(classification_report(y_test, y_pred_rf, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_rf, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_rf))
print('AUC:', roc_auc_score(y_test, y_pred_prob_rf))

              precision    recall  f1-score   support

           0       0.91      0.97      0.94      8517
           1       0.32      0.12      0.18       912

    accuracy                           0.89      9429
   macro avg       0.62      0.55      0.56      9429
weighted avg       0.86      0.89      0.87      9429

Precision: 0.3237822349570201
Accuracy: 0.8902322621699014
AUC: 0.8127066944542287


In [22]:
param_grid= {
             'n_estimators': [100, 200, 300],
             'max_depth': [None, 10, 20, 30],
             'min_samples_split': [2, 5, 10],
             'min_samples_leaf': [1, 2, 4],
            }

gs_tuned_rf= GridSearchCV(estimator= rf, param_grid= param_grid, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_tuned_rf.fit(X_sm_tl, y_sm_tl)
print("Best Parameters:", gs_tuned_rf.best_params_)
print("Best Precision Score:", gs_tuned_rf.best_score_)

Fitting 5 folds for each of 108 candidates, totalling 540 fits
Best Parameters: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 200}
Best Precision Score: 0.9747355169512488


In [23]:
tuned_rf= gs_tuned_rf.best_estimator_
tuned_rf.fit(X_sm_tl_selected, y_sm_tl)
y_pred_tuned_rf= tuned_rf.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_rf= tuned_rf.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_rf, labels= tuned_rf.classes_)
print(classification_report(y_test, y_pred_tuned_rf, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_rf, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_rf))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_rf))

              precision    recall  f1-score   support

           0       0.91      0.98      0.94      8517
           1       0.39      0.14      0.20       912

    accuracy                           0.90      9429
   macro avg       0.65      0.56      0.57      9429
weighted avg       0.86      0.90      0.87      9429

Precision: 0.3888888888888889
Accuracy: 0.895641107222399
AUC: 0.8290183693500512


--- AdaBoost ---

In [24]:
from sklearn.ensemble import AdaBoostClassifier
ada= AdaBoostClassifier(random_state=15)
ada.fit(X_sm_tl_selected, y_sm_tl)
y_pred_ada= ada.predict(X_sm_tl_test_selected)
y_pred_prob_ada= ada.predict_proba(X_sm_tl_test_selected)[:,1]

In [25]:
cm= confusion_matrix(y_test, y_pred_ada, labels= ada.classes_)
print(classification_report(y_test, y_pred_ada, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_ada, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_ada))
print('AUC:', roc_auc_score(y_test, y_pred_prob_ada))

              precision    recall  f1-score   support

           0       0.94      0.92      0.93      8517
           1       0.37      0.45      0.41       912

    accuracy                           0.87      9429
   macro avg       0.66      0.69      0.67      9429
weighted avg       0.89      0.87      0.88      9429

Precision: 0.37488626023657873
Accuracy: 0.8741117827977516
AUC: 0.8428915517777655


In [26]:
param_grid= {'n_estimators': [50, 100, 200]}

gs_ada= GridSearchCV(estimator= ada, param_grid= param_grid, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_ada.fit(X_sm_tl, y_sm_tl)
print("Best Parameters:", gs_ada.best_params_)
print("Best Precision Score:", gs_ada.best_score_)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Best Parameters: {'n_estimators': 200}
Best Precision Score: 0.9611136817860977


In [27]:
tuned_ada= gs_ada.best_estimator_
tuned_ada.fit(X_sm_tl_selected, y_sm_tl)
y_pred_tuned_ada= tuned_ada.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_ada= tuned_ada.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_ada, labels= tuned_ada.classes_)
print(classification_report(y_test, y_pred_tuned_ada, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_ada, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_ada))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_ada))

              precision    recall  f1-score   support

           0       0.92      0.96      0.94      8517
           1       0.44      0.27      0.34       912

    accuracy                           0.90      9429
   macro avg       0.68      0.62      0.64      9429
weighted avg       0.88      0.90      0.89      9429

Precision: 0.44404332129963897
Accuracy: 0.8967016650758299
AUC: 0.84362376897392


--- GradientBoosting ---

In [28]:
from sklearn.ensemble import GradientBoostingClassifier
grb= GradientBoostingClassifier(random_state=15)
grb.fit(X_sm_tl_selected, y_sm_tl)
y_pred_grb= grb.predict(X_sm_tl_test_selected)
y_pred_prob_grb= grb.predict_proba(X_sm_tl_test_selected)[:,1]


cm= confusion_matrix(y_test, y_pred_grb, labels= grb.classes_)
print(classification_report(y_test, y_pred_grb, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_grb, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_grb))
print('AUC:', roc_auc_score(y_test, y_pred_prob_grb))

              precision    recall  f1-score   support

           0       0.93      0.95      0.94      8517
           1       0.39      0.32      0.36       912

    accuracy                           0.89      9429
   macro avg       0.66      0.64      0.65      9429
weighted avg       0.88      0.89      0.88      9429

Precision: 0.39466666666666667
Accuracy: 0.8865203096828932
AUC: 0.8422232547289321


In [29]:
param_grid= {'learning_rate': [0.01, 0.1, 0.2]}

gs_grb= GridSearchCV(estimator= grb, param_grid= param_grid, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_grb.fit(X_sm_tl, y_sm_tl)

print("Best Parameters:", gs_grb.best_params_)
print("Best Precision Score:", gs_grb.best_score_)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Best Parameters: {'learning_rate': 0.2}
Best Precision Score: 0.971352205142504


In [30]:
tuned_grb= gs_grb.best_estimator_
tuned_grb.fit(X_sm_tl_selected, y_sm_tl)
y_pred_tuned_grb= tuned_grb.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_grb= tuned_grb.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_grb, labels= tuned_grb.classes_)
print(classification_report(y_test, y_pred_tuned_grb, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_grb, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_grb))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_grb))

              precision    recall  f1-score   support

           0       0.92      0.98      0.95      8517
           1       0.48      0.21      0.29       912

    accuracy                           0.90      9429
   macro avg       0.70      0.59      0.62      9429
weighted avg       0.88      0.90      0.88      9429

Precision: 0.48009950248756217
Accuracy: 0.901580231201612
AUC: 0.842971242757004


--- XGB ---

In [31]:
from xgboost import XGBClassifier
xgb= XGBClassifier(random_state=15)
xgb.fit(X_sm_tl_selected, y_sm_tl)
y_pred_xgb= xgb.predict(X_sm_tl_test_selected)
y_pred_prob_xgb= xgb.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_xgb, labels= xgb.classes_)
print(classification_report(y_test, y_pred_xgb, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_xgb, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_xgb))
print('AUC:', roc_auc_score(y_test, y_pred_prob_xgb))

              precision    recall  f1-score   support

           0       0.91      0.98      0.95      8517
           1       0.45      0.13      0.20       912

    accuracy                           0.90      9429
   macro avg       0.68      0.56      0.57      9429
weighted avg       0.87      0.90      0.87      9429

Precision: 0.44696969696969696
Accuracy: 0.9003075617774949
AUC: 0.8305943582391461


In [32]:
params_XGBoost= {'learning_rate': [0.01, 0.1, 1.0]}

gs_xgb= GridSearchCV(estimator= xgb, param_grid= params_XGBoost, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_xgb.fit(X_sm_tl, y_sm_tl)

print("Best Parameters:", gs_xgb.best_params_)
print("Best Precision Score:", gs_xgb.best_score_)

tuned_xgb= gs_xgb.best_estimator_
tuned_xgb.fit(X_sm_tl_selected, y_sm_tl)
y_pred_tuned_xgb= tuned_xgb.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_xgb= tuned_xgb.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_xgb, labels= tuned_xgb.classes_)
print(classification_report(y_test, y_pred_tuned_xgb, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_xgb, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_xgb))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_xgb))

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Best Parameters: {'learning_rate': 0.1}
Best Precision Score: 0.9687437596374279
              precision    recall  f1-score   support

           0       0.92      0.97      0.95      8517
           1       0.46      0.21      0.29       912

    accuracy                           0.90      9429
   macro avg       0.69      0.59      0.62      9429
weighted avg       0.88      0.90      0.88      9429

Precision: 0.46454767726161367
Accuracy: 0.9002015059921519
AUC: 0.8394197801507409


--- LGBM ---

In [33]:
from lightgbm import LGBMClassifier
lgm= LGBMClassifier(random_state=15)
lgm.fit(X_sm_tl_selected, y_sm_tl)
y_pred_lgm= lgm.predict(X_sm_tl_test_selected)
y_pred_prob_lgm= lgm.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_lgm, labels= lgm.classes_)
print(classification_report(y_test, y_pred_lgm, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_lgm, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_lgm))
print('AUC:', roc_auc_score(y_test, y_pred_prob_lgm))

[LightGBM] [Info] Number of positive: 33993, number of negative: 33993
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.018570 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 7298
[LightGBM] [Info] Number of data points in the train set: 67986, number of used features: 33
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
              precision    recall  f1-score   support

           0       0.92      0.98      0.95      8517
           1       0.50      0.16      0.24       912

    accuracy                           0.90      9429
   macro avg       0.71      0.57      0.60      9429
weighted avg       0.88      0.90      0.88      9429

Precision: 0.4965986394557823
Accuracy: 0.9030650121964153
AUC: 0.8427464601241274


In [34]:
params_LGB= {'learning_rate': [0.001, 0.01, 0.1, 1.0],
             'num_leaves': [31, 127],
             'reg_alpha': [0.1, 0.5],
             'min_data_in_leaf': [30, 50, 100, 300, 400]}

gs_lgm= GridSearchCV(estimator= lgm, param_grid= params_LGB, cv=5, scoring='precision', n_jobs=-1, verbose=2)
gs_lgm.fit(X_sm_tl, y_sm_tl)

print("Best Parameters:", gs_lgm.best_params_)
print("Best Precision Score:", gs_lgm.best_score_)

tuned_lgm= gs_lgm.best_estimator_
tuned_lgm.fit(X_sm_tl_selected, y_sm_tl)
y_pred_tuned_lgm= tuned_lgm.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_lgm= tuned_lgm.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_lgm, labels= tuned_lgm.classes_)
print(classification_report(y_test, y_pred_tuned_lgm, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_lgm, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_lgm))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_lgm))

Fitting 5 folds for each of 80 candidates, totalling 400 fits
[LightGBM] [Warning] min_data_in_leaf is set=300, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=300
[LightGBM] [Warning] min_data_in_leaf is set=300, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=300
[LightGBM] [Info] Number of positive: 33993, number of negative: 33993
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.013112 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 8572
[LightGBM] [Info] Number of data points in the train set: 67986, number of used features: 42
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
Best Parameters: {'learning_rate': 0.1, 'min_data_in_leaf': 300, 'num_leaves': 31, 'reg_alpha': 0.1}
Best Precision Score: 0.9800382832044241
[LightGBM] [Warning] min_data_in_leaf is set=

--- CatBoost ---

In [35]:
from catboost import CatBoostClassifier
cat= CatBoostClassifier(random_state=15)
cat.fit(X_sm_tl_selected, y_sm_tl)
y_pred_cat= cat.predict(X_sm_tl_test_selected)
y_pred_prob_cat= cat.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_cat, labels= cat.classes_)
print(classification_report(y_test, y_pred_cat, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_cat, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_cat))
print('AUC:', roc_auc_score(y_test, y_pred_prob_cat))

Learning rate set to 0.062428
0:	learn: 0.6247399	total: 164ms	remaining: 2m 43s
1:	learn: 0.5682234	total: 189ms	remaining: 1m 34s
2:	learn: 0.5233968	total: 215ms	remaining: 1m 11s
3:	learn: 0.4875266	total: 238ms	remaining: 59.3s
4:	learn: 0.4559994	total: 265ms	remaining: 52.7s
5:	learn: 0.4342196	total: 288ms	remaining: 47.8s
6:	learn: 0.4100777	total: 313ms	remaining: 44.3s
7:	learn: 0.3855731	total: 336ms	remaining: 41.7s
8:	learn: 0.3718480	total: 360ms	remaining: 39.7s
9:	learn: 0.3574625	total: 384ms	remaining: 38s
10:	learn: 0.3450935	total: 407ms	remaining: 36.6s
11:	learn: 0.3310703	total: 434ms	remaining: 35.7s
12:	learn: 0.3249298	total: 460ms	remaining: 34.9s
13:	learn: 0.3154414	total: 487ms	remaining: 34.3s
14:	learn: 0.3067642	total: 515ms	remaining: 33.8s
15:	learn: 0.2990302	total: 548ms	remaining: 33.7s
16:	learn: 0.2909952	total: 573ms	remaining: 33.1s
17:	learn: 0.2842849	total: 598ms	remaining: 32.6s
18:	learn: 0.2788075	total: 620ms	remaining: 32s
19:	learn: 0

162:	learn: 0.1570447	total: 4.06s	remaining: 20.9s
163:	learn: 0.1568638	total: 4.08s	remaining: 20.8s
164:	learn: 0.1567659	total: 4.11s	remaining: 20.8s
165:	learn: 0.1566639	total: 4.13s	remaining: 20.7s
166:	learn: 0.1565781	total: 4.15s	remaining: 20.7s
167:	learn: 0.1565048	total: 4.17s	remaining: 20.7s
168:	learn: 0.1564414	total: 4.2s	remaining: 20.6s
169:	learn: 0.1563901	total: 4.22s	remaining: 20.6s
170:	learn: 0.1562985	total: 4.24s	remaining: 20.6s
171:	learn: 0.1562237	total: 4.26s	remaining: 20.5s
172:	learn: 0.1561039	total: 4.29s	remaining: 20.5s
173:	learn: 0.1560339	total: 4.31s	remaining: 20.5s
174:	learn: 0.1559640	total: 4.33s	remaining: 20.4s
175:	learn: 0.1558900	total: 4.35s	remaining: 20.4s
176:	learn: 0.1556697	total: 4.37s	remaining: 20.3s
177:	learn: 0.1555987	total: 4.39s	remaining: 20.3s
178:	learn: 0.1555134	total: 4.42s	remaining: 20.3s
179:	learn: 0.1554494	total: 4.44s	remaining: 20.2s
180:	learn: 0.1552661	total: 4.46s	remaining: 20.2s
181:	learn: 0

326:	learn: 0.1402132	total: 8.67s	remaining: 17.8s
327:	learn: 0.1401683	total: 8.7s	remaining: 17.8s
328:	learn: 0.1401151	total: 8.74s	remaining: 17.8s
329:	learn: 0.1400893	total: 8.78s	remaining: 17.8s
330:	learn: 0.1400171	total: 8.81s	remaining: 17.8s
331:	learn: 0.1399699	total: 8.84s	remaining: 17.8s
332:	learn: 0.1399156	total: 8.88s	remaining: 17.8s
333:	learn: 0.1398377	total: 8.91s	remaining: 17.8s
334:	learn: 0.1397818	total: 8.95s	remaining: 17.8s
335:	learn: 0.1397164	total: 8.98s	remaining: 17.7s
336:	learn: 0.1395206	total: 9.01s	remaining: 17.7s
337:	learn: 0.1394727	total: 9.05s	remaining: 17.7s
338:	learn: 0.1394322	total: 9.09s	remaining: 17.7s
339:	learn: 0.1393801	total: 9.12s	remaining: 17.7s
340:	learn: 0.1393297	total: 9.15s	remaining: 17.7s
341:	learn: 0.1392704	total: 9.19s	remaining: 17.7s
342:	learn: 0.1390959	total: 9.23s	remaining: 17.7s
343:	learn: 0.1387077	total: 9.28s	remaining: 17.7s
344:	learn: 0.1386635	total: 9.32s	remaining: 17.7s
345:	learn: 0

490:	learn: 0.1306937	total: 14.5s	remaining: 15s
491:	learn: 0.1306766	total: 14.5s	remaining: 15s
492:	learn: 0.1306419	total: 14.6s	remaining: 15s
493:	learn: 0.1306096	total: 14.6s	remaining: 14.9s
494:	learn: 0.1305580	total: 14.6s	remaining: 14.9s
495:	learn: 0.1305074	total: 14.7s	remaining: 14.9s
496:	learn: 0.1304630	total: 14.7s	remaining: 14.9s
497:	learn: 0.1304282	total: 14.8s	remaining: 14.9s
498:	learn: 0.1303903	total: 14.8s	remaining: 14.9s
499:	learn: 0.1302842	total: 14.8s	remaining: 14.8s
500:	learn: 0.1302264	total: 14.9s	remaining: 14.8s
501:	learn: 0.1301720	total: 14.9s	remaining: 14.8s
502:	learn: 0.1301484	total: 14.9s	remaining: 14.8s
503:	learn: 0.1301112	total: 15s	remaining: 14.7s
504:	learn: 0.1300653	total: 15s	remaining: 14.7s
505:	learn: 0.1300120	total: 15s	remaining: 14.7s
506:	learn: 0.1299626	total: 15.1s	remaining: 14.7s
507:	learn: 0.1299234	total: 15.1s	remaining: 14.6s
508:	learn: 0.1298770	total: 15.2s	remaining: 14.6s
509:	learn: 0.1298436	to

652:	learn: 0.1236131	total: 20.2s	remaining: 10.7s
653:	learn: 0.1235929	total: 20.2s	remaining: 10.7s
654:	learn: 0.1235635	total: 20.3s	remaining: 10.7s
655:	learn: 0.1235246	total: 20.3s	remaining: 10.6s
656:	learn: 0.1234962	total: 20.3s	remaining: 10.6s
657:	learn: 0.1234588	total: 20.4s	remaining: 10.6s
658:	learn: 0.1234222	total: 20.4s	remaining: 10.6s
659:	learn: 0.1233778	total: 20.4s	remaining: 10.5s
660:	learn: 0.1233552	total: 20.5s	remaining: 10.5s
661:	learn: 0.1233415	total: 20.5s	remaining: 10.5s
662:	learn: 0.1232948	total: 20.5s	remaining: 10.4s
663:	learn: 0.1232711	total: 20.6s	remaining: 10.4s
664:	learn: 0.1232389	total: 20.6s	remaining: 10.4s
665:	learn: 0.1231961	total: 20.7s	remaining: 10.4s
666:	learn: 0.1231761	total: 20.7s	remaining: 10.3s
667:	learn: 0.1231415	total: 20.7s	remaining: 10.3s
668:	learn: 0.1231190	total: 20.8s	remaining: 10.3s
669:	learn: 0.1230844	total: 20.8s	remaining: 10.3s
670:	learn: 0.1230416	total: 20.8s	remaining: 10.2s
671:	learn: 

813:	learn: 0.1180972	total: 26.5s	remaining: 6.06s
814:	learn: 0.1180612	total: 26.6s	remaining: 6.03s
815:	learn: 0.1180156	total: 26.6s	remaining: 6s
816:	learn: 0.1179368	total: 26.6s	remaining: 5.97s
817:	learn: 0.1178982	total: 26.7s	remaining: 5.93s
818:	learn: 0.1178618	total: 26.7s	remaining: 5.9s
819:	learn: 0.1178299	total: 26.7s	remaining: 5.87s
820:	learn: 0.1178035	total: 26.8s	remaining: 5.84s
821:	learn: 0.1177804	total: 26.8s	remaining: 5.8s
822:	learn: 0.1177592	total: 26.8s	remaining: 5.77s
823:	learn: 0.1177224	total: 26.9s	remaining: 5.74s
824:	learn: 0.1177021	total: 26.9s	remaining: 5.71s
825:	learn: 0.1176787	total: 27s	remaining: 5.68s
826:	learn: 0.1176436	total: 27s	remaining: 5.65s
827:	learn: 0.1176051	total: 27s	remaining: 5.62s
828:	learn: 0.1175685	total: 27.1s	remaining: 5.58s
829:	learn: 0.1175475	total: 27.1s	remaining: 5.55s
830:	learn: 0.1175189	total: 27.1s	remaining: 5.52s
831:	learn: 0.1174996	total: 27.2s	remaining: 5.49s
832:	learn: 0.1174419	t

973:	learn: 0.1131173	total: 32.3s	remaining: 861ms
974:	learn: 0.1130827	total: 32.3s	remaining: 828ms
975:	learn: 0.1130488	total: 32.3s	remaining: 795ms
976:	learn: 0.1130277	total: 32.4s	remaining: 762ms
977:	learn: 0.1130065	total: 32.4s	remaining: 729ms
978:	learn: 0.1129804	total: 32.5s	remaining: 696ms
979:	learn: 0.1129559	total: 32.5s	remaining: 663ms
980:	learn: 0.1129098	total: 32.5s	remaining: 630ms
981:	learn: 0.1128775	total: 32.6s	remaining: 597ms
982:	learn: 0.1128555	total: 32.6s	remaining: 564ms
983:	learn: 0.1128155	total: 32.6s	remaining: 531ms
984:	learn: 0.1127878	total: 32.7s	remaining: 497ms
985:	learn: 0.1127628	total: 32.7s	remaining: 464ms
986:	learn: 0.1127277	total: 32.7s	remaining: 431ms
987:	learn: 0.1127185	total: 32.8s	remaining: 398ms
988:	learn: 0.1126990	total: 32.8s	remaining: 365ms
989:	learn: 0.1126590	total: 32.8s	remaining: 332ms
990:	learn: 0.1126299	total: 32.9s	remaining: 298ms
991:	learn: 0.1125911	total: 32.9s	remaining: 265ms
992:	learn: 

In [36]:
params_CatBoost= {
                  'depth': [3,5,10],
                  'learning_rate' : [0.01,0.1,1],
                  'iterations' : [5,10,50,100]
                 }

gs_cat= GridSearchCV(estimator= cat, param_grid= params_CatBoost, cv=5, scoring='precision', n_jobs=-1, verbose=2)
gs_cat.fit(X_sm_tl, y_sm_tl)

print("Best Parameters:", gs_cat.best_params_)
print("Best Precision Score:", gs_cat.best_score_)

tuned_cat= gs_cat.best_estimator_
tuned_cat.fit(X_sm_tl_selected, y_sm_tl)
y_pred_tuned_cat= tuned_cat.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_cat= tuned_cat.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_cat, labels= tuned_cat.classes_)
print(classification_report(y_test, y_pred_tuned_cat, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_cat, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_cat))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_cat))

Fitting 5 folds for each of 36 candidates, totalling 180 fits
0:	learn: 0.4923625	total: 19.1ms	remaining: 934ms
1:	learn: 0.4102545	total: 43.6ms	remaining: 1.05s
2:	learn: 0.3467355	total: 66.4ms	remaining: 1.04s
3:	learn: 0.2992550	total: 90.1ms	remaining: 1.04s
4:	learn: 0.2826226	total: 109ms	remaining: 982ms
5:	learn: 0.2584881	total: 133ms	remaining: 972ms
6:	learn: 0.2322352	total: 154ms	remaining: 944ms
7:	learn: 0.2252604	total: 173ms	remaining: 908ms
8:	learn: 0.2194321	total: 195ms	remaining: 888ms
9:	learn: 0.2123800	total: 215ms	remaining: 859ms
10:	learn: 0.1999490	total: 235ms	remaining: 832ms
11:	learn: 0.1915375	total: 257ms	remaining: 815ms
12:	learn: 0.1821783	total: 279ms	remaining: 794ms
13:	learn: 0.1800626	total: 296ms	remaining: 761ms
14:	learn: 0.1727670	total: 318ms	remaining: 743ms
15:	learn: 0.1672289	total: 338ms	remaining: 718ms
16:	learn: 0.1632210	total: 357ms	remaining: 693ms
17:	learn: 0.1622734	total: 378ms	remaining: 673ms
18:	learn: 0.1586862	total

--- Visualisation ---

In [37]:
precision_scores= {
                    'Logistic Regression Precision:': precision_score(y_test, y_pred_lr, zero_division= 0),
                    'Decision Tree Precision:': precision_score(y_test, y_pred_dt, zero_division= 0),
                    'Tuned Decision Tree Precision:': precision_score(y_test, y_pred_tuned_dt, zero_division= 0),
                    'KNeighborsClassifier Precision:': precision_score(y_test, y_pred_knn, zero_division= 0),
                    'Tuned KNeighborsClassifier Precision:': precision_score(y_test, y_pred_tuned_knn, zero_division= 0),
                    'GaussianNB Precision:': precision_score(y_test, y_pred_nb, zero_division= 0),
                    'SVM Precision:': precision_score(y_test, y_pred_svc, zero_division= 0),
                    'Random Forest Precision:': precision_score(y_test, y_pred_rf, zero_division= 0),
                    'Tuned Random Forest Precision:': precision_score(y_test, y_pred_tuned_rf, zero_division= 0),
                    'AdaBoost Precision:': precision_score(y_test, y_pred_ada, zero_division= 0),
                    'Tuned AdaBoost Precision:': precision_score(y_test, y_pred_tuned_ada, zero_division= 0),
                    'GradientBoosting Precision:': precision_score(y_test, y_pred_grb, zero_division= 0),
                    'Tuned GradientBoosting Precision:': precision_score(y_test, y_pred_tuned_grb, zero_division= 0),
                    'XGB Precision:': precision_score(y_test, y_pred_xgb, zero_division= 0),
                    'Tuned XGB Precision:': precision_score(y_test, y_pred_tuned_xgb, zero_division= 0),
                    'LGBM Precision:': precision_score(y_test, y_pred_lgm, zero_division= 0),
                    'Tuned LGBM Precision:': precision_score(y_test, y_pred_tuned_lgm, zero_division= 0),
                    'CatBoost Precision:': precision_score(y_test, y_pred_cat, zero_division= 0),
                    'Tuned CatBoost Precision:': precision_score(y_test, y_pred_tuned_cat, zero_division= 0)
                  }

be_precision= pd.DataFrame(list(precision_scores.items()), columns= ['Model', 'Precision Score'])
be_precision= be_precision.sort_values(by= 'Precision Score', ascending=False)
print(be_precision)

                                    Model  Precision Score
18              Tuned CatBoost Precision:         0.534483
16                  Tuned LGBM Precision:         0.505495
15                        LGBM Precision:         0.496599
17                    CatBoost Precision:         0.487805
12      Tuned GradientBoosting Precision:         0.480100
14                   Tuned XGB Precision:         0.464548
13                         XGB Precision:         0.446970
10              Tuned AdaBoost Precision:         0.444043
11            GradientBoosting Precision:         0.394667
8          Tuned Random Forest Precision:         0.388889
9                     AdaBoost Precision:         0.374886
7                Random Forest Precision:         0.323782
2          Tuned Decision Tree Precision:         0.286127
0          Logistic Regression Precision:         0.256374
4   Tuned KNeighborsClassifier Precision:         0.246356
1                Decision Tree Precision:         0.2357

In [38]:
accuracy_scores= {
                    'Logistic Regression Accuracy:': accuracy_score(y_test, y_pred_lr),
                    'Decision Tree Accuracy:': accuracy_score(y_test, y_pred_dt),
                    'Tuned Decision Tree Accuracy:': accuracy_score(y_test, y_pred_tuned_dt),
                    'KNeighborsClassifier Accuracy:': accuracy_score(y_test, y_pred_knn),
                    'Tuned KNeighborsClassifier Accuracy:': accuracy_score(y_test, y_pred_tuned_knn),
                    'GaussianNB Accuracy:': accuracy_score(y_test, y_pred_nb),
                    'SVM Accuracy:': accuracy_score(y_test, y_pred_svc),
                    'Random Forest Accuracy:': accuracy_score(y_test, y_pred_rf),
                    'Tuned Random Forest Accuracy:': accuracy_score(y_test, y_pred_tuned_rf),
                    'AdaBoost Accuracy:': accuracy_score(y_test, y_pred_ada),
                    'Tuned AdaBoost Accuracy:': accuracy_score(y_test, y_pred_tuned_ada),
                    'GradientBoosting Accuracy:': accuracy_score(y_test, y_pred_grb),
                    'Tuned GradientBoosting Accuracy:': accuracy_score(y_test, y_pred_tuned_grb),
                    'XGB Accuracy:': accuracy_score(y_test, y_pred_xgb),
                    'Tuned XGB Accuracy:': accuracy_score(y_test, y_pred_tuned_xgb),
                    'LGBM Accuracy:': accuracy_score(y_test, y_pred_lgm),
                    'Tuned LGBM Accuracy:': accuracy_score(y_test, y_pred_tuned_lgm),
                    'CatBoost Accuracy:': accuracy_score(y_test, y_pred_cat),
                    'Tuned CatBoost Accuracy:': accuracy_score(y_test, y_pred_tuned_cat)
                  }

be_accuracy= pd.DataFrame(list(accuracy_scores.items()), columns= ['Model', 'Accuracy Score'])
be_accuracy= be_accuracy.sort_values(by= 'Accuracy Score', ascending=False)
print(be_accuracy)

                                   Model  Accuracy Score
18              Tuned CatBoost Accuracy:        0.905398
16                  Tuned LGBM Accuracy:        0.903595
15                        LGBM Accuracy:        0.903065
17                    CatBoost Accuracy:        0.902641
12      Tuned GradientBoosting Accuracy:        0.901580
13                         XGB Accuracy:        0.900308
14                   Tuned XGB Accuracy:        0.900202
10              Tuned AdaBoost Accuracy:        0.896702
8          Tuned Random Forest Accuracy:        0.895641
7                Random Forest Accuracy:        0.890232
11            GradientBoosting Accuracy:        0.886520
6                          SVM Accuracy:        0.877081
9                     AdaBoost Accuracy:        0.874112
2          Tuned Decision Tree Accuracy:        0.871885
4   Tuned KNeighborsClassifier Accuracy:        0.866370
1                Decision Tree Accuracy:        0.842401
3         KNeighborsClassifier 

In [39]:
auc_scores= {
                    'Logistic Regression AUC:': roc_auc_score(y_test, y_pred_prob_lr),
                    'Decision Tree AUC:': roc_auc_score(y_test, y_pred_prob_dt),
                    'Tuned Decision Tree AUC:': roc_auc_score(y_test, y_pred_prob_tuned_dt),
                    'KNeighborsClassifier AUC:': roc_auc_score(y_test, y_pred_prob_knn),
                    'Tuned KNeighborsClassifier AUC:': roc_auc_score(y_test, y_pred_prob_tuned_knn),
                    'GaussianNB AUC:': roc_auc_score(y_test, y_pred_prob_nb),
                    'SVM AUC:': roc_auc_score(y_test, y_pred_prob_svc),
                    'Random Forest AUC:': roc_auc_score(y_test, y_pred_prob_rf),
                    'Tuned Random Forest AUC:': roc_auc_score(y_test, y_pred_prob_tuned_rf),
                    'AdaBoost AUC:': roc_auc_score(y_test, y_pred_prob_ada),
                    'Tuned AdaBoost AUC:': roc_auc_score(y_test, y_pred_prob_tuned_ada),
                    'GradientBoosting AUC:': roc_auc_score(y_test, y_pred_prob_grb),
                    'Tuned GradientBoosting AUC:': roc_auc_score(y_test, y_pred_prob_tuned_grb),
                    'XGB AUC:': roc_auc_score(y_test, y_pred_prob_xgb),
                    'Tuned XGB AUC:': roc_auc_score(y_test, y_pred_prob_tuned_xgb),
                    'LGBM AUC:': roc_auc_score(y_test, y_pred_prob_lgm),
                    'Tuned LGBM AUC:': roc_auc_score(y_test, y_pred_prob_tuned_lgm),
                    'CatBoost AUC:': roc_auc_score(y_test, y_pred_prob_cat),
                    'Tuned CatBoost AUC:': roc_auc_score(y_test, y_pred_prob_tuned_cat)
                  }

be_auc= pd.DataFrame(list(auc_scores.items()), columns= ['Model', 'AUC Score'])
be_auc= be_auc.sort_values(by= 'AUC Score', ascending=False)
print(be_auc)

                              Model  AUC Score
0          Logistic Regression AUC:   0.851767
10              Tuned AdaBoost AUC:   0.843624
12      Tuned GradientBoosting AUC:   0.842971
9                     AdaBoost AUC:   0.842892
15                        LGBM AUC:   0.842746
16                  Tuned LGBM AUC:   0.842657
11            GradientBoosting AUC:   0.842223
18              Tuned CatBoost AUC:   0.840369
14                   Tuned XGB AUC:   0.839420
17                    CatBoost AUC:   0.836067
13                         XGB AUC:   0.830594
8          Tuned Random Forest AUC:   0.829018
7                Random Forest AUC:   0.812707
5                   GaussianNB AUC:   0.786267
3         KNeighborsClassifier AUC:   0.739891
2          Tuned Decision Tree AUC:   0.722918
6                          SVM AUC:   0.714756
4   Tuned KNeighborsClassifier AUC:   0.621157
1                Decision Tree AUC:   0.590974
